In [17]:
import os
# hugging face镜像设置，如果国内环境无法使用启用该设置
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import dotenv 
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings


In [18]:
# 本地markdown文件路径
markdown_path = "../../data/C1/markdown/easy-rl-chapter1.md"

# 加载本地markdown文件
loader = UnstructuredMarkdownLoader(markdown_path)
docs = loader.load()

# print(docs)

# 文本分块
text_splitter = RecursiveCharacterTextSplitter()
chunks = text_splitter.split_documents(docs)

In [19]:
# 文本切分
embeddings = OllamaEmbeddings(
    model="bge-m3:latest"
)
# 构建向量存储
vectorstore = InMemoryVectorStore(embeddings)
vectorstore.add_documents(chunks)

['4205cd68-abe3-4ff1-8c84-2b69ef01aaeb',
 '933328e0-8acc-405a-a740-f558c5330e34',
 '25da1a07-708c-4cfd-af98-96d28fb34765',
 '4600c93c-37d0-4992-9ca6-efb585cec7f2',
 '8d071ef9-ce42-4cb7-a43f-569ef7d5b1b8',
 'e0ecf919-b7a7-42d3-8c88-be6e3e253b16']

In [20]:

# 提示词模板
prompt = ChatPromptTemplate.from_template("""请根据下面提供的上下文信息来回答问题。
请确保你的回答完全基于这些上下文。
如果上下文中没有足够的信息来回答问题，请直接告知：“抱歉，我无法根据提供的上下文找到相关信息来回答此问题。”

上下文:
{context}

问题: {question}

回答:"""
                                          )



In [30]:

dotenv.load_dotenv(override=True)
# 配置大语言模型
llm = ChatOpenAI(
    model="qwen-plus",
    temperature=0.7,
    max_tokens=2048,
    base_url='https://dashscope.aliyuncs.com/compatible-mode/v1',
    api_key=os.getenv('DASHSCOPE_API_KEY')
    # api_key="sk-b1e8a6140c6a4fb2b7ae64b7074a09e6"

)


In [29]:

# 用户查询
question = "文中举了哪些例子？"

# 在向量存储中查询相关文档
retrieved_docs = vectorstore.similarity_search(question, k=3)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

answer = llm.invoke(prompt.format(question=question, context=docs_content))
print(answer.content)

文中举了以下例子：

1. **雅达利游戏 Breakout**：用于说明强化学习中的时间序列数据和延迟奖励问题。
2. **Pong 游戏**：作为强化学习的经典例子，说明智能体如何通过试错学习策略，并与监督学习进行对比。
3. **股票交易**：将买卖股票的过程视为一个强化学习过程，通过市场反馈来学习最大化奖励的策略。
4. **自然界中的羚羊**：刚出生的羚羊通过试错学会站立和奔跑，适应环境，类比强化学习的探索过程。
5. **AlphaGo**：强化学习算法在围棋中战胜人类顶尖棋手，展示强化学习具有超越人类表现的潜力。
6. **选择餐馆**：用日常决策比喻探索（尝试新餐馆）与利用（去最喜欢的餐馆）的权衡。
7. **做广告**：利用指采用已知最优广告策略，探索指尝试新策略以寻求更好效果。
8. **挖油**：利用是在已知油田的地方开采，探索是在新地点钻探，可能失败也可能发现大油田。
9. **玩游戏（《街头霸王》）**：利用是重复使用某个有效策略，探索是尝试新招式，可能打出“大招”。
10. **MountainCar-v0（小车上山）**：作为 Gym 库中的具体示例，演示如何实现智能体与环境交互，包括观测空间、动作空间以及智能体决策逻辑。

这些例子涵盖了自然现象、日常生活、游戏和金融等多个领域，用于帮助理解强化学习的核心概念。


In [32]:
from langchain_text_splitters  import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../../data/C2/txt/蜂医.txt")
docs = loader.load()

text_splitter = CharacterTextSplitter(
    chunk_size=200,    # 每个块的目标大小为100个字符
    chunk_overlap=10   # 每个块之间重叠10个字符，以缓解语义割裂
)

chunks = text_splitter.split_documents(docs)

print(f"文本被切分为 {len(chunks)} 个块。\n")
print("--- 前5个块内容示例 ---")
for i, chunk in enumerate(chunks[:5]):
    print("=" * 60)
    # chunk 是一个 Document 对象，需要访问它的 .page_content 属性来获取文本
    print(f'块 {i+1} (长度: {len(chunk.page_content)}): "{chunk.page_content}"')

Created a chunk of size 201, which is longer than the specified 200
Created a chunk of size 277, which is longer than the specified 200
Created a chunk of size 296, which is longer than the specified 200


文本被切分为 14 个块。

--- 前5个块内容示例 ---
块 1 (长度: 72): "# 蜂医

游戏《三角洲行动》中的支援型干员

蜂医是2024年琳琅天上发行的《三角洲行动》中的支援型干员之一，在早期版本是唯一一个支援型干员。"
块 2 (长度: 201): "蜂医在游戏中能够使用战术装备“激素枪”：远程治疗队友或'自我治疗'，还可以使用兵种道具“烟幕无人机”：释放长烟幕，和“蜂巢科技烟雾弹”：产生一团白色烟雾（使用激素枪对烟雾射击换变成绿色烟雾，可起到治疗作用），干员特长为“高效救援”：救援倒地队友时速度更快，在全面战场模式中约1.4秒就能救起队友，且被救起的队友能恢复更多生命值。在烽火地带中，还能够移除队友血量上限减少的负面效果。 \[1-2]****"
块 3 (长度: 189): "* 中文名

  罗伊•斯米

* 外文名

  Roy smee \[2]**

* 别    名

  罗伊、蜂医

* 性    别

  男

- 登场作品

  [三角洲行动](/item/%E4%B8%89%E8%A7%92%E6%B4%B2%E8%A1%8C%E5%8A%A8/63251542?fromModule=lemma_inlink)

- 生    日"
块 4 (长度: 133): "- 生    日

  2008年2月23日 \[3]**

- 身    高

  176 cm \[3]**

- 体    重

  75 kg \[3]**

## 目录

1. 1[角色设定](#1)
2. 2[角色定位](#2)
3. 3[技能](#3)"
块 5 (长度: 189): "1) ▪[战术装备 - 激素枪](#3-1)
2) ▪[战术道具 - 烟幕无人机](#3-2)
3) ▪[战术道具 - 蜂巢科技烟雾弹](#3-3)

1. ▪[干员特长 - 高效救援](#3-4)

## 角色设定

三角洲行动：医疗角色蜂医！让你不再为血量担忧

蜂医是游戏中的一名战地医生，拥有丰富的参军履历。

蜂医在干员档案中标明他有一个妻子和女儿。

## 角色定位"


In [33]:
from langchain_experimental.text_splitters.SemanticChunker import SemanticChunker

ModuleNotFoundError: No module named 'langchain_experimental.text_splitters'

In [34]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings

# 1. 示例文本和嵌入模型
texts = [
    "张三是法外狂徒",
    "FAISS是一个用于高效相似性搜索和密集向量聚类的库。",
    "LangChain是一个用于开发由语言模型驱动的应用程序的框架。"
]
docs = [Document(page_content=t) for t in texts]
embeddings=OllamaEmbeddings(
    model="bge-m3:latest"
)

# 2. 创建向量存储并保存到本地
vectorstore = FAISS.from_documents(docs, embeddings)

local_faiss_path = "./faiss_index_store"
vectorstore.save_local(local_faiss_path)

print(f"FAISS index has been saved to {local_faiss_path}")

# 3. 加载索引并执行查询
# 加载时需指定相同的嵌入模型，并允许反序列化
loaded_vectorstore = FAISS.load_local(
    local_faiss_path,
    embeddings,
    allow_dangerous_deserialization=True
)

# 相似性搜索
query = "FAISS是做什么的？"
results = loaded_vectorstore.similarity_search(query, k=1)

print(f"\n查询: '{query}'")
print("相似度最高的文档:")
for doc in results:
    print(f"- {doc.page_content}")

FAISS index has been saved to ./faiss_index_store

查询: 'FAISS是做什么的？'
相似度最高的文档:
- FAISS是一个用于高效相似性搜索和密集向量聚类的库。
